In [1]:
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u
import astrodendro
import os
from astropy import wcs
from astrodendro import Dendrogram, pp_catalog
from astrodendro.analysis import PPStatistic
from astropy.table import Table, hstack, Column
from astropy.utils.console import ProgressBar
import reproject
import pyregion
print(np.__version__)
import aplpy

1.16.5


In [229]:
# Image information and assumptions
distance        = 8150. # distance to GC; Reid et al. 2019
#Temp            = 20.0
Wave            = (3.0e8/226.e9)
Wave0           = 1.3e-3
k0              = 0.899
nu              = 3.e08/Wave
nu0             = 3.e08/Wave0
beta            = 1.75
Kappag2d        = k0*((nu/nu0)**beta)
g2d             = 100.0
Kappa           = Kappag2d / g2d
mu              = 2.8 # express everything in H2

dustTemp = 20

# Constants
G = 6.67408e-11
msun = 1.989e33
mh = 1.6737236e-27
pc2cm = 3.08567758e18
as2persr = 4.25e10 
percm2perm = 1.0e6
JyperSr_to_JyperPix = (3.0462*(10**-4))*((0.00013888888888)**2)
JyperSr_to_Jyperassqr = ((np.pi*180.)**(-2))*(3600**(-1))
hplanck = 6.63e-34
clight = 2.99792e8
kboltzmann = 1.381e-23
sin1yr = 3.15569e7
arcsec2pc = distance/((360./(2.*np.pi))*60.*60.)

### Define physical property functions

def planck_wave( Wave, Temp ):

    # Planck function using wavelength
    #output in Jy/Sr

    planck_conv_wave = 1.e-26 * clight / Wave**2.0

    planck = ((2.0*hplanck*clight**2.0)/(Wave**5.0))*(1.0/(np.exp((hplanck*clight)/(Wave*kboltzmann*Temp))-1.0))
    planck = planck/planck_conv_wave

    return planck

def mass_calc_submm( Wave, Temp, Kappa, Integrated_Flux, Obj_Dist ):
    
    Obj_Dist = Obj_Dist * pc2cm
    B = planck_wave( Wave, Temp )
    Mass = (Obj_Dist**2. * Integrated_Flux) / ( Kappa * B )
    Mass = Mass / msun
    return Mass

def column_density(Wave, Temp, Kappa, Flux, mu):
    
    B = planck_wave( Wave, Temp )
    N = Flux / (mu * (mh*1.e3) * Kappa * B)
    return N

def number_density_sphere_pc( Mass_sol, Radius_pc, mu ):

    # This subroutine accepts mass in solar masses and radius in pc and calculates the number density.

    Mass = Mass_sol * (msun/1000.0)
    print('Mass in kg is'+str(Mass[0]))
    Radius = Radius_pc * (pc2cm/100.0)
    print('radius in m is'+str(Radius[0]))

    n = Mass / (((4. / 3.)*np.pi) * mu * mh * (Radius**3.0))

    # Convert to particles per cubic centimetre

    n = n / percm2perm

    return n

def mass_density_sphere( Mass_sol, Radius_pc ):

    # This subroutine accepts mass in solar masses and radius in pc and calculates the mass density.
    # returns mass density in kg/m**3 ??
    Mass = Mass_sol * (msun/1000.0)
    Radius = Radius_pc * (pc2cm/100.0)

    rho = Mass / (((4. / 3.)*np.pi) * Radius**3.0)

    return rho

def tff_spherical_F( number_density, mu ):

    # Accepts a number density in units of particles per cubic centimetre and derives the free fall time in yrs

    mass_density = mu * mh * number_density * percm2perm

    tff = np.sqrt( (3. * np.pi) / (32. * G * mass_density) )

    tff = tff / sin1yr # free-fall time in years

    return tff

In [204]:

def mass_err(I_int, I_peak_err, Td, Td_err, d, d_err, area, Kappa):
    '''
    I_int: integrated flux in Janskys
    I_peak_err: The peak flux uncertainty in Janskys per Steradian
    Td: dust temperature in Kelvin
    Td_err: dust temperature uncertainty in Kelvin
    d: distance to galactic center in parsecs
    d_err: distance uncertainty in parsecs
    npix: size of object in square arcsec
    Kappa: assumed dust opacity in cm^2 /gram
    '''
    d = d * pc2cm
    d_err = d_err * pc2cm
    I_int_err = I_peak_err*(area/(3600*3600*3282.8)) #convert area to sr 
    B = planck_wave(Wave,Td)
    dterm = (2*d*I_int*d_err)/(Kappa*B)
    Iterm = (I_int_err*d**2)/(Kappa*B)
    Tterm = Td_err * ((I_int * d**2)/Kappa) * (((1.0e-26)*(Wave**2))/(2*kboltzmann*(Td**2))) *np.exp(hplanck*clight/(kboltzmann*Wave*Td))
    return np.sqrt(dterm**2 + Iterm**2 + Tterm**2)/msun


def N_err(I_peak, I_peak_err, Td, Td_err, Kappa):
    '''
    I_peak: peak flux in Janskys
    I_peak_err: The peak flux uncertainty in Janskys per Steradian
    Td: dust temperature in Kelvin
    Td_err: dust temperature uncertainty in Kelvin
    Kappa: assumed dust opacity in cm^2 /gram
    '''
    B = planck_wave(Wave,Td)
    Iterm = (I_peak_err)/(mu * (mh*1.e3) * Kappa * B)
    Tterm = Td_err * (I_peak/(mu * (mh*1.e3) * Kappa)) * (((1.0e-26)*(Wave**2))/(2*kboltzmann*(Td**2))) *np.exp(hplanck*clight/(kboltzmann*Wave*Td))
    return np.sqrt(Iterm**2 + Tterm**2)

def ndens_err(I_int, I_peak_err, Td, Td_err, d, d_err, area, Kappa, Radius_pc):
    """
    I_int: integrated flux in Janskys
    I_peak_err: The peak flux uncertainty in Janskys per Steradian
    Td: dust temperature in Kelvin
    Td_err: dust temperature uncertainty in Kelvin
    d: distance to galactic center in parsecs
    d_err: distance uncertainty in parsecs
    npix: size of object in square arcsec
    Kappa: assumed dust opacity in cm^2 /gram
    Radius_pc: effective radius in parsecs
    """
    Radius = Radius_pc * (pc2cm/100.0) #convert to SI
    m_err = mass_err(I_int=catalog['flux_integrated'],I_peak_err=catalog['noise'],
                     Td=20.0,Td_err=5.0,
                     d=8150.0,d_err=150.0,
                     area=catalog['area_exact'],Kappa=Kappa)
    Mass = m_err*msun / (1000.0) #convert to SI
    n_err = Mass/(((4./3.)*np.pi) * mu * mh * Radius**3.0)
    n_err = n_err / percm2perm
    return n_err

def rho_err(I_int, I_peak_err, Td, Td_err, d, d_err, area, Kappa, Radius_pc):
    """
    I_int: integrated flux in Janskys
    I_peak_err: The peak flux uncertainty in Janskys per Steradian
    Td: dust temperature in Kelvin
    Td_err: dust temperature uncertainty in Kelvin
    d: distance to galactic center in parsecs
    d_err: distance uncertainty in parsecs
    npix: size of object in square arcsec
    Kappa: assumed dust opacity in cm^2 /gram
    Radius_pc: effective radius in parsecs
    returns mass density in 
    """
    Radius = Radius_pc * (pc2cm/100.0) #convert to SI
    m_err = mass_err(I_int=catalog['flux_integrated'],I_peak_err=catalog['noise'],
                     Td=20.0,Td_err=5.0,
                     d=8150.0,d_err=150.0,
                     area=catalog['area_exact'],Kappa=Kappa)
    Mass = m_err*msun / (1000.0) #convert to cgs
    rho_err = Mass/(((4./3.)*np.pi) * Radius**3.0)

    return rho_err
    
def tff_err(I_int, I_peak_err, Td, Td_err, d, d_err, area, Kappa, Radius_pc, mass_density):
    """
    I_int: integrated flux in Janskys
    I_peak_err: The peak flux uncertainty in Janskys per Steradian
    Td: dust temperature in Kelvin
    Td_err: dust temperature uncertainty in Kelvin
    d: distance to galactic center in parsecs
    d_err: distance uncertainty in parsecs
    npix: size of object in square arcsec
    Kappa: assumed dust opacity in cm^2 /gram
    Radius_pc: effective radius in parsecs
    returns freefall time error in years
    """
    number_density_err = ndens_err(I_int, I_peak_err, Td, Td_err, d, d_err, area, Kappa, Radius_pc)
    mass_density_err = mu * mh * number_density_err * percm2perm
    sigma_tff = np.sqrt((3. * np.pi) / (32. * G * mass_density**3))*mass_density_err
    sigma_tff = sigma_tff / sin1yr # free-fall time in years

    return sigma_tff



In [205]:
path = '/Users/hph/cmzoom_catalog/catalog_acc/'

catalog = Table.read(path+'catalog_acc.fits')
mass = catalog['mass']
column=catalog['N']

In [208]:
sigma_mass = mass_err(I_int=catalog['flux_integrated'],I_peak_err=catalog['noise'],
                      Td=20.0,Td_err=5.0,
                      d=8150.0,d_err=150.0,
                      area=catalog['area_exact'],Kappa=Kappa)
sigma_ndens = ndens_err(I_int=catalog['flux_integrated'],I_peak_err=catalog['noise'],
                      Td=20.0,Td_err=5.0,
                      d=8150.0,d_err=150.0,
                      area=catalog['area_exact'],Kappa=Kappa,Radius_pc=catalog['r_eff_pc'])
sigma_rho = rho_err(I_int=catalog['flux_integrated'],I_peak_err=catalog['noise'],
                      Td=20.0,Td_err=5.0,
                      d=8150.0,d_err=150.0,
                      area=catalog['area_exact'],Kappa=Kappa,Radius_pc=catalog['r_eff_pc'])
sigma_column = N_err(I_peak=catalog['peak_cont_flux'],I_peak_err=catalog['noise'],
                     Td=20.0,Td_err=5.0,Kappa=Kappa)
sigma_tff = tff_err(I_int=catalog['flux_integrated'],I_peak_err=catalog['noise'],
                    Td=20.0,Td_err=5.0,
                    d=8150.0,d_err=150.0,
                    area=catalog['area_exact'],Kappa=Kappa,Radius_pc=catalog['r_eff_pc'],
                    mass_density=catalog['rho'])

In [199]:
sigma_mass/catalog['mass']

0.4010921724837639
0.31247189091290367
0.3370508222589929
0.3893343131478624
0.36149738039336715
0.3456392752045843
0.31243835917027735
0.3589862927230043
0.3201724054991591
0.3458493314521572
0.3551165732590167


In [200]:
sigma_column/catalog['N']

0.34579650412363266
0.3239088406598332
0.3320221996841026
0.35684976232799154
0.351628470561884
0.3410224280817021
0.32397767952808576
0.3586929363246828
0.3316088994992853
0.3547687341643716
0.3593822075017465


In [201]:
sigma_ndens/catalog['n']

0.40109217248376394
0.31247189091290356
0.3370508222589929
0.38933431314786243
0.36149738039336715
0.3456392752045843
0.31243835917027735
0.35898629272300425
0.3201724054991591
0.3458493314521572
0.3551165732590167


In [209]:
sigma_tff/catalog['tff']

0.4010921724837639
0.31247189091290356
0.3370508222589928
0.3893343131478624
0.3614973803933672
0.3456392752045844
0.31243835917027735
0.3589862927230044
0.3201724054991591
0.3458493314521572
0.3551165732590168


In [210]:
catalog

index,area_ellipse,area_exact,flux_integrated,major_sigma,minor_sigma,position_angle,radius,glon,glat,r_eff,testflux_integrated,flux_integrated_bgsub,min_flux_integrated,noise,is_leaf,peak_cont_flux,min_cont_flux,mean_cont_flux,npix,ColumnDensity,mass,N,r_eff_pc,Sigma,n,rho,tff,glon_0,mass_bgsub,leaf_ID,mask_num
,arcsec2,arcsec2,Jy,arcsec,arcsec,deg,arcsec,deg,deg,arcsec,,,,,,Jy / sr,,Jy / sr,,,solMass,1 / cm2,arcsec,g / cm2,1 / cm3,kg / m3,yr,deg,solMass,,
int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bool,float64,float64,float64,int64,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,bytes16,int64
99,15.93642962736847,44.99999999424,0.06711277017465443,3.066618642541715,1.1932551629971624,49.98987387633183,1.9129188503844723,-0.3760651010894094,-0.24710558434732224,3.7846987827880194,0.06711333864267492,0.027139740230957166,0.039973029943697266,16808116.604430154,True,138673398.29364792,37792110.252965584,63451649.699837305,180,6.8058127e+22,109.17243059895792,1.4456421197345135e+23,0.15302847064777161,0.24196053329536812,105062.81506253393,4.923691165472759e-16,94869.17736580166,0.3760651010894094,44.148250759538456,G359.611+0.018c,33
118,50.27924382752407,259.249999966816,2.092034161041695,3.432449652509003,3.363462780211621,176.63735384078174,3.397781136442479,-0.3853654338704473,-0.24323223654332535,9.084153124126923,2.0920518812835387,1.7412478515626204,0.3507863094790745,9878086.472014785,True,1726650427.7703958,57566583.85239814,343320924.42923653,1037,1.4556968e+23,3403.114692816353,1.799998136020899e+24,0.3673037511036135,3.012699360214189,236839.0240518991,1.1099285790785672e-15,63186.313787548344,0.3853654338704473,2832.4901465935286,G359.611+0.018a,33
122,11.752823560482662,34.749999995551995,0.0751624231646436,2.1722544885214017,1.2423202874156973,115.3938641262996,1.642752513472418,-0.3820674204332022,-0.24458589102278241,3.3258485448785673,0.07516305981596644,0.028137800631797168,0.04702462253284643,12460174.199866636,True,170279866.42564774,57572766.26793404,92022966.7286485,139,8.594786e+22,122.26681159542541,1.7751331551450054e+23,0.13447556744633804,0.2971082254908657,173392.8551894198,8.125927986453603e-16,73847.203247259,0.3820674204332022,45.771796912688046,G359.611+0.018b,33
325,10.911462084169491,38.74999999504,0.07120138534213535,1.8678774336666697,1.3413330850424916,69.820126613889,1.582860006627676,0.3234758680090484,-0.20806974240770998,3.512051834475549,0.07120198844211881,0.03413491203885718,0.037066473303278176,19145497.214292657,True,127757236.51297668,40696414.49183426,78174840.38944133,155,3.8607202e+22,115.8233862137543,1.3318433418134758e+23,0.1420044108951738,0.22291376326960816,139490.14376819826,6.537102476582338e-16,82333.74885710164,-0.3234758680090484,55.52730584455951,G0.316-0.201d,17
374,5.974909031462293,18.999999997568,0.038750434477579525,1.366342110256029,1.0040929284762714,76.39833879435797,1.1712960559941388,0.3228069094690509,-0.2037324220074243,2.459245379525576,0.03875076270695708,0.010806064769426863,0.02794436970815266,16704207.79900477,True,121963054.86846082,62573047.25888247,86770728.11102815,76,4.6472244e+22,63.03538220332149,1.2714401704931376e+23,0.099435802153631,0.21280394193423882,221110.21829299658,1.0362166935627922e-15,65395.10548148151,-0.3228069094690509,17.578239625901016,G0.316-0.201e,17
381,4.899790680031683,15.499999998016,0.03133792748019114,1.3372103455354436,0.8413561308392384,-173.24764493631287,1.0606932272989689,0.3147805618240951,-0.2034411771308461,2.221216611503081,0.031338192923109895,0.009006715799519373,0.022331211680671766,13536097.053467894,True,126709999.80412915,61295300.420524105,86017900.74476513,62,4.0083406e+22,50.977447422344596,1.3209261110087883e+23,0.08981147524383588,0.2210865205851629,242680.76900369427,1.1373054849733684e-15,62421.179399947025,-0.3147805618240951,14.651236314469857,G0.316-0.201g,17
400,27.059886629555166,115.2

In [219]:
#####################
### fix units
#####################

print(sigma_mass.unit,sigma_column.unit,sigma_ndens.unit,sigma_rho.unit,sigma_tff.unit)


Jy None Jy Jy kg / m3


In [237]:
#################
# add unc_tables
#################
path = '/Users/hph/cmzoom_catalog/catalog_acc/'

catalog = Table.read(path+'catalog_acc.fits')
catalog.add_columns(cols=[sigma_mass,sigma_column,sigma_ndens,sigma_rho,sigma_tff],
                    names=['mass_unc','N_unc','n_unc','rho_unc','tff_unc'])
catalog['mass_unc'].unit=u.solMass
catalog['N_unc'].unit = u.cm**(-2)
catalog['n_unc'].unit = 1./u.cm**3
catalog['rho_unc'].unit = u.kg/u.m**3
catalog['tff_unc'].unit = u.s
catalog.write(path+'catalog_acc.fits',overwrite=True)

In [235]:
catalog

index,area_ellipse,area_exact,flux_integrated,major_sigma,minor_sigma,position_angle,radius,glon,glat,r_eff,testflux_integrated,flux_integrated_bgsub,min_flux_integrated,noise,is_leaf,peak_cont_flux,min_cont_flux,mean_cont_flux,npix,ColumnDensity,mass,N,r_eff_pc,Sigma,n,rho,tff,glon_0,mass_bgsub,leaf_ID,mask_num,mass_unc,N_unc,n_unc,rho_unc,tff_unc
,arcsec2,arcsec2,Jy,arcsec,arcsec,deg,arcsec,deg,deg,arcsec,,,,,,Jy / sr,,Jy / sr,,,solMass,1 / cm2,arcsec,g / cm2,1 / cm3,kg / m3,yr,deg,solMass,,,solMass,1 / cm2,1 / cm3,kg / m3,s
int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bool,float64,float64,float64,int64,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,bytes16,int64,float64,float64,float64,float64,float64
99,15.93642962736847,44.99999999424,0.06711277017465443,3.066618642541715,1.1932551629971624,49.98987387633183,1.9129188503844723,-0.3760651010894094,-0.24710558434732224,3.7846987827880194,0.06711333864267492,0.027139740230957166,0.039973029943697266,16808116.604430154,True,138673398.29364792,37792110.252965584,63451649.699837305,180,6.8058127e+22,109.17243059895792,1.4456421197345135e+23,0.15302847064777161,0.24196053329536812,105062.81506253393,4.923691165472759e-16,94869.17736580166,0.3760651010894094,44.148250759538456,G359.611+0.018c,33,43.78820736426897,4.998979912180728e+22,42139.87274069165,1.9748539861985843e-16,38051.28445139691
118,50.27924382752407,259.249999966816,2.092034161041695,3.432449652509003,3.363462780211621,176.63735384078174,3.397781136442479,-0.3853654338704473,-0.24323223654332535,9.084153124126923,2.0920518812835387,1.7412478515626204,0.3507863094790745,9878086.472014785,True,1726650427.7703958,57566583.85239814,343320924.42923653,1037,1.4556968e+23,3403.114692816353,1.799998136020899e+24,0.3673037511036135,3.012699360214189,236839.0240518991,1.1099285790785672e-15,63186.313787548344,0.3853654338704473,2832.4901465935286,G359.611+0.018a,33,1063.377683057811,5.830353094283902e+23,74005.53768746355,3.4682148188295212e-16,19743.9469490113
122,11.752823560482662,34.749999995551995,0.0751624231646436,2.1722544885214017,1.2423202874156973,115.3938641262996,1.642752513472418,-0.3820674204332022,-0.24458589102278241,3.3258485448785673,0.07516305981596644,0.028137800631797168,0.04702462253284643,12460174.199866636,True,170279866.42564774,57572766.26793404,92022966.7286485,139,8.594786e+22,122.26681159542541,1.7751331551450054e+23,0.13447556744633804,0.2971082254908657,173392.8551894198,8.125927986453603e-16,73847.203247259,0.3820674204332022,45.771796912688046,G359.611+0.018b,33,41.2101293832235,5.893836149034261e+22,58442.20441542842,2.738850709451549e-16,24890.26057601561
325,10.911462084169491,38.74999999504,0.07120138534213535,1.8678774336666697,1.3413330850424916,69.820126613889,1.582860006627676,0.3234758680090484,-0.20806974240770998,3.512051834475549,0.07120198844211881,0.03413491203885718,0.037066473303278176,19145497.214292657,True,127757236.51297668,40696414.49183426,78174840.38944133,155,3.8607202e+22,115.8233862137543,1.3318433418134758e+23,0.1420044108951738,0.22291376326960816,139490.14376819826,6.537102476582338e-16,82333.74885710164,-0.3234758680090484,55.52730584455951,G0.316-0.201d,17,45.09401851799162,4.752679799842568e+22,54308.299314888056,2.5451183026973746e-16,32055.353560168267
374,5.974909031462293,18.999999997568,0.038750434477579525,1.366342110256029,1.0040929284762714,76.39833879435797,1.1712960559941388,0.3228069094690509,-0.2037324220074243,2.459245379525576,0.03875076270695708,0.010806064769426863,0.02794436970815266,16704207.79900477,True,121963054.86846082,62573047.25888247,86770728.11102815,76,4.6472244e+22,63.03538220332149,1.2714401704931376e+23,0.099435802153631,0.21280394193423882,221110.21829299658,1.0362166935627922e-15,65395.10548148151,-0.3228069094690509,17.578239625901016,G0.316-0.201e,17,22.787125538595394,4.47074562561443e+22,79930.76469112383,3.745896202428

In [232]:
catalog['rho']*1000*1.0e6

4.92369116547276e-07
1.1099285790785671e-06
8.125927986453603e-07
6.537102476582338e-07
1.0362166935627922e-06
1.1373054849733684e-06
1.6331903732025074e-06
1.3364307893837076e-06
2.4549459872269206e-06
1.6212804397171537e-06
6.774119239365371e-07
